In [14]:
import numpy as np

Реализовать переводчик используя ноутбук HW-3-colab(в материалах) / в колабе (можно и локально, но тогда надо поправить его немного)
(проще делать в колабе)

In [ ]:
!sudo pip install -q --upgrade nltk bokeh pandas

import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.manifold import TSNE

In [ ]:
# !pip install gensim  --upgrade --ignore-installed

In [ ]:
# !sudo pip install numpy --upgrade --ignore-installed

In [ ]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from string import punctuation

In [71]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
!wget -O ukr_rus.train.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1vAK0SWXUqei4zTimMvIhH3ufGPsbnC_O"
!wget -O ukr_rus.test.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1W9R2F8OeKHXruo2sicZ6FgBJUTJc8Us_"
!wget -O fairy_tale.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1sq8zSroFeg_afw-60OmY8RATdu_T1tej"

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id': '1d7OXuil646jUeDS1JNhP9XWlZogv6rbu'})
downloaded.GetContentFile('cc.ru.300.vec.zip')

downloaded = drive.CreateFile({'id': '1yAqwqgUHtMSfGS99WLGe5unSCyIXfIxi'})
downloaded.GetContentFile('cc.uk.300.vec.zip')

!unzip cc.ru.300.vec.zip
!unzip cc.uk.300.vec.zip

Archive:  cc.ru.300.vec.zip
  inflating: cc.ru.300.vec           
Archive:  cc.uk.300.vec.zip
  inflating: cc.uk.300.vec           


In [ ]:
from gensim.models import KeyedVectors

ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

In [15]:
ru_emb.most_similar([ru_emb["август"]])

[('август', 1.0000001192092896),
 ('июль', 0.9383152723312378),
 ('сентябрь', 0.9240029454231262),
 ('июнь', 0.9222574830055237),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545795679092407),
 ('февраль', 0.8401415944099426)]

In [16]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r", encoding='utf8') as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)


uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

In [20]:
uk_ru_train[0]

('iснує', 'существует')

In [22]:
from sklearn.linear_model import LinearRegression
mapping = LinearRegression(fit_intercept=False)
mapping.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None, normalize=False)

In [49]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8541284799575806),
 ('июнь', 0.841120183467865),
 ('март', 0.8396992683410645),
 ('сентябрь', 0.835986852645874),
 ('февраль', 0.8329296112060547),
 ('октябрь', 0.8311845660209656),
 ('ноябрь', 0.8278923034667969),
 ('июль', 0.8234528303146362),
 ('август', 0.8120500445365906),
 ('декабрь', 0.8039003610610962)]

Должно получиться, что в топе содержатся разные месяцы, но август не первый.

Будем мерять percision top-k с k = 1, 5, 10.

**Задание** Реализуйте следующую функцию:

In [27]:
for i,j in enumerate([("серпень", "август")]):
    print(i, j)

0 ('серпень', 'август')


In [28]:
august[0]

array([ 2.30992399e-03, -4.31192592e-02, -4.31668684e-02, -3.46036106e-02,
       -1.71317905e-03, -3.56488526e-02, -4.91967425e-03,  1.53636895e-02,
        2.78865248e-02,  3.03796530e-02, -3.84306721e-03,  4.43296731e-02,
       -1.11899115e-02,  1.83284879e-02,  1.90534815e-02, -2.39052512e-02,
        5.20489402e-02,  1.65656209e-03, -7.67123178e-02,  3.39718498e-02,
        4.69692200e-02, -5.40798753e-02, -3.05023193e-02, -3.28376889e-03,
        4.58409935e-02,  5.08170053e-02,  3.82830761e-03, -3.13807204e-02,
        3.07917818e-02, -4.78136539e-03, -2.07080618e-02,  1.61701329e-02,
        2.16257926e-02, -8.49262401e-02,  1.41674206e-02, -1.57319270e-02,
       -5.04165776e-02, -1.71213359e-01, -3.32581401e-02,  1.82292163e-02,
        3.49867158e-03, -4.00891192e-02, -3.91736440e-03, -1.25834830e-02,
       -1.46666374e-02,  2.60908389e-03,  3.46445404e-02,  8.69558603e-02,
        3.34554538e-03,  5.36744483e-02,  3.27022709e-02,  1.89813040e-02,
        2.82948390e-02,  

In [54]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    ru_vectors = mapped_vectors
    assert len(pairs) == len(ru_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        to_translate = ru_emb.most_similar(mapped_vectors[i])
        to_translate = to_translate[: topn]
        to_translate = [i[0] for i in to_translate]
        if ru in to_translate:
            num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val

In [53]:
ru_emb.most_similar(Y_test[0])

[('либо', 1.0000001192092896),
 ('Либо', 0.7894865870475769),
 ('или', 0.736264705657959),
 ('-либо', 0.6304555535316467),
 ('вовсе', 0.5090411901473999),
 ('каких-', 0.5040323734283447),
 ('Или', 0.5023478865623474),
 ('нет.Если', 0.4999190866947174),
 ('т.е.', 0.49034079909324646),
 ('просто-напросто', 0.4899868965148926)]

In [55]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [56]:
# assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [57]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.813

In [46]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    matrix = np.dot(X_train.T, Y_train)
    U, S, V = np.linalg.svd(matrix)
    
    return np.dot(U, V)

In [47]:
W = learn_transform(X_train, Y_train)

In [48]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8237906694412231),
 ('сентябрь', 0.8049713373184204),
 ('март', 0.8025653958320618),
 ('июнь', 0.8021842241287231),
 ('октябрь', 0.8001735806465149),
 ('ноябрь', 0.7934483289718628),
 ('февраль', 0.7914120554924011),
 ('июль', 0.7908109426498413),
 ('август', 0.7891016602516174),
 ('декабрь', 0.7686373591423035)]

In [58]:
assert precision(uk_ru_test, np.matmul(X_test, W)) >= 0.653
assert precision(uk_ru_test, np.matmul(X_test, W), 5) >= 0.824

In [59]:
# Сингулярные прелести

In [67]:
with open("fairy_tale.txt", "r") as f:
    uk_sentences = [line.rstrip().lower() for line in f]

In [68]:
len(uk_sentences)

67

In [70]:
uk_sentences[0].split()

['лисичка', '-', 'сестричка', 'і', 'вовк', '-', 'панібрат']

In [84]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    list_ukr = sentence.split()
    
    rus_list = []
    for word in list_ukr:
        try:
            if int(word):
                rus_list.append(word)
        except:   
            if word in punctuation:
                rus_list.append(word)
            else:
                try:
                    sim = ru_emb.most_similar([np.matmul(uk_emb[word], W)])[0][0]
                
                    rus_list.append(sim)
                except:
                    rus_list.append(word)

    return ' '.join(rus_list)

In [85]:
translate("1 , 3")

'1 , 3'

In [86]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

In [87]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

src: лисичка - сестричка і вовк - панібрат
dst: лисичка - сестричка и волк - панібрат

src: як була собі лисичка , да й пішла раз до однії баби добувать огню ; ввійшла у хату да й каже : " добрий день тобі , бабусю !
dst: как была себе лисичка , че и пошла раз к однії бабы добувать огня ; вошла во избу че и говорит : " хороший день тебе , бабушку !

src: дай мені огня " .
dst: дай мне огня " .

src: а баба тільки що вийняла із печі пирожок із маком , солодкий , да й положила , щоб він прохолов ; а лисичка се і підгледала , да тілько що баба нахилилась у піч , щоб достать огня , то лисичка зараз ухватила пирожок да і драла з хати , да , біжучи , весь мак із його виїла , а туда сміття наклала .
dst: а бабка только что вынула со печи пирожок со маком , сладкий , че и согнула , чтобы он прохолов ; а лисичка ой и підгледала , че токмо что бабка качнулась во печь , чтобы достать огня , то лисичка сейчас ухватила пирожок че и деру со хаты , че , пробежать , весь мак со его виїла , а туда мусо